<a href="https://colab.research.google.com/github/yudintsev-sergey/Neyro/blob/main/CNN_music_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключаем диск Google My Drive

In [ ]:
from google.colab import drive
drive.mount("/ShardDrives/")

Mounted at /ShardDrives/


Подотовка баз для валидации, обучения и тестирования нейросети, которая
должна опознавать музыкальные сэмплы в трех классах "музыка без голоса", "соло голосом а-капелла", "хоровое многоголосие".
- импортируем необходимые библиотеки, в т.ч. librosa для работы с аудиофайлами
- засекаем время старта программы
- меняем директорию на My Drive

In [ ]:
import librosa
import os
import numpy as np
import math as ma
import time

start = time.time()

os.chdir('/ShardDrives/MyDrive/Colab Notebooks/Try/')

Размерность общей базы определяется длиной вектора мел-кепстральных коэффициентов (16x16 = 256) + вектор признаков (3)

In [ ]:
common_base = np.empty([0, 259], dtype=float)

- Функция 'file_to_base' из аудиофайла сначала получает массив типа 'numpy array'
- Затем из этого файла нарезаются короткие (1.4 сек) сэмплы
- Далее - сэмплы свертываются до матрицы мел-кепстральных коэффициентов (характеристика тембра и вообще индивидуальности звука) размерности (16x16)
- Далее - матрица "вытягивается" (reshape) в вектор-строку [256]
- Далее - к вектору справа добавляется признак в виде [i, j, k], где i + j + k = 1, {i,j,k}={0,1}

In [ ]:
def file_to_base(filename, threeplet=[[1., 0., 0.]]):
    global common_base
    y, sr = librosa.load(filename)
    drtn = librosa.get_duration(y=y, sr=sr)

    off_s = 0.
    while off_s < (drtn - 2):
        y, sr = librosa.load(filename, offset=off_s, duration=1.4)
        tmbr = librosa.feature.mfcc(y=y, sr=sr, n_fft=512, hop_length=2048, n_mfcc=16)
        tmbr_lin = np.append(tmbr.reshape(1, 256), threeplet, axis=1)
        common_base = np.append(common_base, tmbr_lin, axis=0)
        off_s += 1.4
    return common_base

- Исполняем процедуру для выбранных файлов. Каждое очередное исполнение наращивает длину общей базы (common_base).
- Файлы 'zero_.wav' - инструментальная музыка, голос человека отсутствует - признак [1., 0., 0.]
- Файлы 'solo_.wav' - пение а-капелла - признак [0., 1., 0.]
- Файлы 'chor_.wav' - хоровое многоголосие - признак [0., 0., 1.]
- Суммарные длительности однотипных файлов примерно равны

In [ ]:
file_to_base('zero_0.wav', [[1., 0., 0.]])
file_to_base('zero_1.wav', [[1., 0., 0.]])
file_to_base('zero_2.wav', [[1., 0., 0.]])
file_to_base('solo_0.wav', [[0., 1., 0.]])
file_to_base('solo_1.wav', [[0., 1., 0.]])
file_to_base('solo_2.wav', [[0., 1., 0.]])
file_to_base('solo_0.wav', [[0., 1., 0.]])
file_to_base('chor_0.wav', [[0., 0., 1.]])
file_to_base('chor_3.wav', [[0., 0., 1.]])

array([[-9.03776367e+02, -9.03776367e+02, -7.22747742e+02, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.76945343e+02, -3.72588043e+02, -3.20996704e+02, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.98167206e+02, -3.41024200e+02, -3.40618622e+02, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-7.88915161e+02, -8.08188660e+02, -8.06488953e+02, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-8.92409851e+02, -9.02881958e+02, -9.11314392e+02, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-9.47231689e+02, -1.03442847e+03, -9.88318848e+02, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

Перемешиваем строки базовой матрицы случайным образом

In [ ]:
np.random.shuffle(common_base)

Строим 3-Д-матрицу мел-кепстральных коэффициентов (характеристики тембра)

In [ ]:
arrays = [common_base[:, :256][i].reshape(16,16) for i in range(np.shape(common_base)[0])]
tmbr_base = np.stack(arrays, axis=0)

Строим матрицу признаков "true", которые индексированы с матрицей МКК

In [ ]:
true_base = common_base[:, 256:]

Сохраняем базы данных

In [ ]:
np.save('tmbr_base', tmbr_base)
np.save('true_base', true_base)

Измеряем время выполнения программы. Выводим справку

In [ ]:
# record end time
end = time.time()
# печатаем длительность работы алгоритма
print("The time of execution of above program is :", (end-start), "сек")

# Справочные данные
print('Размерность общего массива = ', np.shape(common_base))
print('Число записей в базе мел-кепстральных к-тов (тембр) = ', np.shape(tmbr_base)[0])
print('Число записей в базе признака "true" = ', np.shape(true_base)[0])
print('Для примера первая запись в тембрах', tmbr_base[0])
print('Для примера первая запись в признаках', true_base[0])
print('Файлы np_array сохранены в родительской папке с расширением *.npy')
